In [1]:
import numpy as np
import json
import smplx
import torch

/home/user/miniconda/envs/textured-avatar/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load the SMPL-X model
model_path = "data"  # Replace with the path to your SMPL-X model files
smplx_model = smplx.create(model_path, model_type='smplx', gender='neutral', flat_hand_mean=True)
# smplx_model.num_expression_coeffs
# handmean = torch.zeros([1,45])
# smplx_model.left_hand_mean = handmean
# smplx_model.right_hand_mean = handmean

In [9]:
smplx_model.left_hand_pose

Parameter containing:
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       requires_grad=True)

In [3]:
# Load Unity vertices from the JSON file
with open("ExportedVertices.json", "r") as f:
    unity_data = json.load(f)

uV = np.array(unity_data['vertices'])

betas = torch.Tensor(unity_data['betas']).unsqueeze(0)
expressions = torch.Tensor(unity_data['expressions']).unsqueeze(0)


In [ ]:
# Extract the default canonical vertices from the SMPL-X model
# handpose = torch.zeros([1,45])


output = smplx_model(betas=betas,
                     expression=expressions,
                    #  left_hand_pose=handpose,
                    #  right_hand_pose=handpose,
                     return_full_pose=True)
canonical_vertices = output.vertices.detach().cpu().numpy().squeeze()
b = canonical_vertices
# b = b[b[:, 1].argsort()]
output.full_pose
# handpose.shape




tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.1117,  0.0429, -0.4164,  0.1088, -0.0660,
         -0.7562, -0.0964, -0.0909, -0.1885, -0.1181,  0.0509, -0.5296, -0.1437,
          0.0552, -0.7049, -0.0192, -0.0923, -0.3379, -0.4570, -0.1963, -0.6255,
         -0.2147, -0.0660, -

In [62]:
uNiq, iduN, idv, count = np.unique(uV[:,:], axis=0, return_index=True, return_counts=True, return_inverse=True)
idu = np.sort(iduN)
u = uNiq[idu]



In [63]:
# Extract Unity vertices as a numpy array
# unity_vertices = np.array(unity_data["vertices"])  # Convert list of lists to numpy array

# Match vertices with a tolerance
tolerance = 1e-4
indices = []
badIndices = []
# print(canonical_vertices.shape)
for i, canon_v in enumerate(b):
    # print(f'({i},{canon_v})')
    # Calculate distances between the current canonical vertex and all Unity vertices
    distances = np.linalg.norm(u - canon_v, axis=1)
    
    # Find the closest Unity vertex
    idx = np.argmin(distances)
    
    # Check if the closest vertex is within the tolerance
    if distances[idx] < tolerance:
        indices.append(idx)
    else:
        indices.append(-1)  # Mark as not found if no close match is found
        badIndices.append(idx)

# # Save the mapping indices
# np.savetxt("vertex_mapping.txt", indices, fmt="%d")

# Print summary
print(f"Mapped {len([i for i in indices if i != -1])}/{len(canonical_vertices)} vertices successfully.")

Mapped 8921/10475 vertices successfully.


In [50]:
indices
# badIndices

[7349,
 7436,
 7451,
 7360,
 7713,
 7712,
 7692,
 7684,
 6191,
 5635,
 5628,
 6147,
 7004,
 6815,
 6736,
 7081,
 6784,
 7076,
 7031,
 3125,
 3038,
 3023,
 3114,
 2761,
 2762,
 2782,
 2790,
 6734,
 8319,
 8318,
 8322,
 8321,
 8237,
 8236,
 8272,
 8275,
 8239,
 8218,
 8250,
 8280,
 7709,
 7827,
 7842,
 7630,
 7518,
 7711,
 7693,
 7533,
 8222,
 8225,
 8184,
 8179,
 7698,
 7622,
 7923,
 7940,
 8177,
 8198,
 8212,
 8182,
 8249,
 8256,
 7626,
 7583,
 7608,
 7627,
 7651,
 7731,
 8138,
 8136,
 8196,
 8202,
 8043,
 8111,
 8143,
 8114,
 8242,
 8201,
 8223,
 8267,
 8023,
 7967,
 7919,
 7975,
 8248,
 8261,
 8219,
 7789,
 7824,
 7841,
 7821,
 7828,
 7550,
 7620,
 7760,
 7699,
 7527,
 7647,
 7689,
 7549,
 7943,
 8039,
 7964,
 7878,
 8215,
 8190,
 8207,
 8205,
 8183,
 4283,
 4839,
 4846,
 4327,
 8191,
 8197,
 8204,
 8283,
 8284,
 8274,
 7862,
 7869,
 7836,
 7874,
 7641,
 7710,
 7652,
 7612,
 7786,
 7666,
 7694,
 7813,
 8109,
 8117,
 8069,
 7888,
 7929,
 7979,
 7954,
 8024,
 7913,
 7809,
 7932,
 7616,